<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/347_WDO_LearningPath_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Workforce Development Orchestrator — Learning Path Matching Utilities

This module is responsible for turning **prioritized skill gaps** into **personalized learning recommendations**. It answers a critical question for both employees and leadership:

> **“Given the risks we see, what should this person actually learn next — and why?”**

The design is intentionally conservative, transparent, and policy-driven. Learning paths are recommended only when there is a clear, defensible match.

---

## 1. Prerequisites Are Treated as Guardrails, Not Suggestions

The `check_prerequisites_met` utility ensures that learning recommendations are **realistic and achievable**.

Key design choices:

* Explicit handling of generic prerequisites (e.g., basic skills)
* Strict checking for role- or skill-specific prerequisites
* No silent assumptions about readiness

This prevents a common failure mode in learning systems:

> Recommending content that employees are not prepared to benefit from.

When prerequisites are not met, the system **downgrades confidence rather than forcing a recommendation**.

---

## 2. Match Scoring Is Deterministic and Explainable

The `calculate_learning_path_match_score` function assigns a **numeric confidence score** to each potential recommendation.

That score is driven by:

* Exact alignment between the learning path’s target skill and the detected gap
* Whether prerequisites are met
* The urgency of the gap (derived earlier from automation risk and role needs)

There is no semantic similarity guessing or embedding-based inference here.
A learning path either **addresses the gap**, or it doesn’t.

This keeps recommendations:

* Defensible
* Auditable
* Easy to tune

---

## 3. Thresholds Control Recommendation Behavior

A recommendation is only produced if the match score exceeds a **configurable threshold**.

This gives leadership a clear control lever:

* Raise the bar to reduce noise
* Lower the bar to encourage broader development

Crucially, this avoids overwhelming employees with marginal suggestions while preserving flexibility as organizational needs change.

---

## 4. Recommendations Are Built Around the Employee, Not the Course

The `match_employee_to_learning_paths` function assembles recommendations that are **employee-centered**:

Each recommendation includes:

* Who it’s for
* What skill it targets
* Why it’s relevant
* How long it will take
* Whether prerequisites are met
* How strong the match is

This makes outputs suitable for:

* Employee-facing dashboards
* Manager discussions
* Executive summaries

There is no ambiguity about *why* something was recommended.

---

## 5. Organization-Wide Matching Is Fair and Consistent

The `match_all_employees_to_learning_paths` utility scales the same logic across the entire workforce.

Key characteristics:

* Gaps are grouped by employee
* Each employee is evaluated independently
* The same policies and thresholds apply everywhere

This ensures fairness and avoids “special treatment” logic that erodes trust.

---

## 6. Learning Is Positioned as Enablement, Not Remediation

One subtle but important design choice is how recommendations are framed:

> “Employee missing skill required for role”

This positions learning as **role enablement**, not performance correction.
That framing matters deeply for morale, adoption, and organizational trust.

---

## Why This Design Works for Leaders

From a leadership perspective, this module ensures:

* Learning investments align directly to risk and role needs
* Recommendations are justifiable and measurable
* Noise is controlled through explicit thresholds
* Progress can be tracked meaningfully

From a system design perspective, it ensures:

* Deterministic behavior
* Easy testing
* Clear policy levers
* No hidden model behavior

---

## Architectural Takeaway

This module demonstrates an essential principle of responsible workforce AI:

> **Learning recommendations should be earned, not inferred.**

By requiring explicit skill alignment, prerequisite checks, and configurable thresholds, the orchestrator ensures that every recommendation can be explained — to employees, managers, and executives alike.




In [ ]:
"""Learning path matching utilities for Workforce Development Orchestrator

Following the pattern: Utilities implement, nodes orchestrate.
These utilities match employees to appropriate learning paths.
"""

from typing import Dict, List, Any, Optional, Set
from config import WorkforceDevelopmentOrchestratorConfig


def check_prerequisites_met(
    employee_skills: Set[str],
    prerequisites: List[str],
    skills_lookup: Dict[str, Dict[str, Any]]
) -> bool:
    """Check if employee has all prerequisites for a learning path"""
    # Handle generic prerequisites (like "basic_computer_skills")
    generic_prereqs = {"basic_computer_skills", "basic_excel", "basic_programming"}

    for prereq in prerequisites:
        if prereq in generic_prereqs:
            # Assume generic prerequisites are met (or could be checked separately)
            continue
        elif prereq not in employee_skills:
            # Missing a skill prerequisite
            return False

    return True


def calculate_learning_path_match_score(
    gap: Dict[str, Any],
    learning_path: Dict[str, Any],
    employee_skills: Set[str],
    skills_lookup: Dict[str, Dict[str, Any]]
) -> float:
    """Calculate match score between a skill gap and a learning path"""
    target_skill = learning_path.get("target_skill")
    gap_skill = gap.get("skill_id")

    # Perfect match if learning path targets the gap skill
    if target_skill == gap_skill:
        base_score = 1.0
    else:
        # No match
        return 0.0

    # Check prerequisites
    prerequisites_met = True
    for course in learning_path.get("courses", []):
        course_prereqs = course.get("prerequisites", [])
        if not check_prerequisites_met(employee_skills, course_prereqs, skills_lookup):
            prerequisites_met = False
            break

    # Reduce score if prerequisites not met
    if not prerequisites_met:
        base_score *= 0.5

    # Boost score based on gap priority
    if gap.get("priority") == "high":
        base_score *= 1.1  # 10% boost
    elif gap.get("priority") == "medium":
        base_score *= 1.05  # 5% boost

    # Cap at 1.0
    return min(1.0, base_score)


def match_employee_to_learning_paths(
    employee: Dict[str, Any],
    employee_gaps: List[Dict[str, Any]],
    learning_paths: List[Dict[str, Any]],
    skills_lookup: Dict[str, Dict[str, Any]],
    config: WorkforceDevelopmentOrchestratorConfig
) -> List[Dict[str, Any]]:
    """Match an employee to appropriate learning paths based on their skill gaps"""
    employee_skills = set(employee.get("current_skills", []))
    recommendations = []

    for gap in employee_gaps:
        gap_skill = gap.get("skill_id")

        # Find learning paths that target this skill
        matching_paths = [
            path for path in learning_paths
            if path.get("target_skill") == gap_skill
        ]

        for path in matching_paths:
            match_score = calculate_learning_path_match_score(
                gap,
                path,
                employee_skills,
                skills_lookup
            )

            # Only recommend if match score meets threshold
            if match_score >= config.learning_path_match_threshold:
                prerequisites_met = check_prerequisites_met(
                    employee_skills,
                    [prereq for course in path.get("courses", []) for prereq in course.get("prerequisites", [])],
                    skills_lookup
                )

                recommendation = {
                    "employee_id": employee["employee_id"],
                    "employee_name": employee.get("name", ""),
                    "learning_path_id": path["learning_path_id"],
                    "learning_path_name": path.get("path_name", ""),
                    "target_skill": path.get("target_skill", ""),
                    "match_score": round(match_score, 2),
                    "priority": gap.get("priority", "medium"),
                    "estimated_completion_weeks": path.get("estimated_completion_weeks", 0),
                    "prerequisites_met": prerequisites_met,
                    "rationale": f"Employee missing {gap.get('skill_name', gap_skill)} skill required for role"
                }

                recommendations.append(recommendation)

    return recommendations


def match_all_employees_to_learning_paths(
    employees: List[Dict[str, Any]],
    skill_gap_analysis: List[Dict[str, Any]],
    learning_paths: List[Dict[str, Any]],
    skills_lookup: Dict[str, Dict[str, Any]],
    config: WorkforceDevelopmentOrchestratorConfig
) -> List[Dict[str, Any]]:
    """Match all employees to learning paths"""
    all_recommendations = []

    # Group gaps by employee
    gaps_by_employee: Dict[str, List[Dict[str, Any]]] = {}
    for gap in skill_gap_analysis:
        employee_id = gap.get("employee_id")
        if employee_id:
            if employee_id not in gaps_by_employee:
                gaps_by_employee[employee_id] = []
            gaps_by_employee[employee_id].append(gap)

    # Match each employee
    employees_lookup = {emp["employee_id"]: emp for emp in employees}

    for employee_id, employee_gaps in gaps_by_employee.items():
        employee = employees_lookup.get(employee_id)
        if not employee:
            continue

        recommendations = match_employee_to_learning_paths(
            employee,
            employee_gaps,
            learning_paths,
            skills_lookup,
            config
        )

        all_recommendations.extend(recommendations)

    return all_recommendations



# Test learning path matching utilities

In [ ]:
"""Test learning path matching utilities

Testing Phase 5: Learning Path Matching Utilities
Following the pattern: Test utilities before building nodes
"""

from pathlib import Path
from agents.workforce_development_orchestrator.utilities.learning_path_matching import (
    check_prerequisites_met,
    calculate_learning_path_match_score,
    match_employee_to_learning_paths,
    match_all_employees_to_learning_paths
)
from agents.workforce_development_orchestrator.utilities.data_loading import (
    load_employees,
    load_learning_paths,
    load_skills,
    build_skills_lookup
)
from config import WorkforceDevelopmentOrchestratorConfig


def test_check_prerequisites_met():
    """Test prerequisite checking"""
    data_dir = Path("agents/data")
    skills = load_skills(data_dir)
    skills_lookup = build_skills_lookup(skills)

    employee_skills = {"excel", "data_entry", "communication"}

    # Test with met prerequisites
    prereqs_met = ["excel"]
    assert check_prerequisites_met(employee_skills, prereqs_met, skills_lookup) == True

    # Test with missing prerequisites
    prereqs_missing = ["python", "sql"]
    assert check_prerequisites_met(employee_skills, prereqs_missing, skills_lookup) == False

    # Test with generic prerequisites (should pass)
    prereqs_generic = ["basic_computer_skills"]
    assert check_prerequisites_met(employee_skills, prereqs_generic, skills_lookup) == True

    print("✅ test_check_prerequisites_met: PASSED")


def test_calculate_learning_path_match_score():
    """Test learning path match score calculation"""
    data_dir = Path("agents/data")
    skills = load_skills(data_dir)
    skills_lookup = build_skills_lookup(skills)
    config = WorkforceDevelopmentOrchestratorConfig()

    # Create test gap
    gap = {
        "skill_id": "ai_tools",
        "priority": "high"
    }

    # Create test learning path
    learning_path = {
        "learning_path_id": "LP001",
        "target_skill": "ai_tools",
        "courses": [
            {"prerequisites": []}
        ]
    }

    employee_skills = {"excel", "data_entry"}

    score = calculate_learning_path_match_score(
        gap,
        learning_path,
        employee_skills,
        skills_lookup
    )

    assert 0.0 <= score <= 1.0
    assert score > 0.0  # Should match since target_skill matches

    print("✅ test_calculate_learning_path_match_score: PASSED")


def test_match_employee_to_learning_paths():
    """Test matching employee to learning paths"""
    data_dir = Path("agents/data")
    employees = load_employees(data_dir)
    learning_paths = load_learning_paths(data_dir)
    skills = load_skills(data_dir)
    skills_lookup = build_skills_lookup(skills)
    config = WorkforceDevelopmentOrchestratorConfig()

    # Get E001 and their gaps
    employee = employees[0]  # E001
    employee_gaps = [
        {
            "skill_id": "ai_tools",
            "skill_name": "AI Tools",
            "priority": "high",
            "gap_type": "missing_future_skill"
        }
    ]

    recommendations = match_employee_to_learning_paths(
        employee,
        employee_gaps,
        learning_paths,
        skills_lookup,
        config
    )

    assert len(recommendations) > 0
    assert all("employee_id" in r for r in recommendations)
    assert all("learning_path_id" in r for r in recommendations)
    assert all("match_score" in r for r in recommendations)

    print("✅ test_match_employee_to_learning_paths: PASSED")


def test_match_all_employees_to_learning_paths():
    """Test matching all employees to learning paths"""
    data_dir = Path("agents/data")
    employees = load_employees(data_dir)
    learning_paths = load_learning_paths(data_dir)
    skills = load_skills(data_dir)
    skills_lookup = build_skills_lookup(skills)
    config = WorkforceDevelopmentOrchestratorConfig()

    # Create sample skill gap analysis
    skill_gap_analysis = [
        {
            "employee_id": "E001",
            "skill_id": "ai_tools",
            "skill_name": "AI Tools",
            "priority": "high"
        },
        {
            "employee_id": "E002",
            "skill_id": "automation_workflows",
            "skill_name": "Automation Workflows",
            "priority": "high"
        }
    ]

    recommendations = match_all_employees_to_learning_paths(
        employees,
        skill_gap_analysis,
        learning_paths,
        skills_lookup,
        config
    )

    assert len(recommendations) > 0
    assert all("match_score" >= config.learning_path_match_threshold for r in recommendations)

    print("✅ test_match_all_employees_to_learning_paths: PASSED")


if __name__ == "__main__":
    print("=" * 60)
    print("Testing Learning Path Matching Utilities (Phase 5)")
    print("=" * 60)
    print()

    test_check_prerequisites_met()
    test_calculate_learning_path_match_score()
    test_match_employee_to_learning_paths()
    test_match_all_employees_to_learning_paths()

    print()
    print("=" * 60)
    print("✅ All learning path utility tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_008_Workforce_Development_Orchestrator % python3 test_learning_path_utilities.py
============================================================
Testing Learning Path Matching Utilities (Phase 5)
============================================================

✅ test_check_prerequisites_met: PASSED
✅ test_calculate_learning_path_match_score: PASSED
✅ test_match_employee_to_learning_paths: PASSED
✅ test_match_all_employees_to_learning_paths: PASSED

============================================================
✅ All learning path utility tests passed!
============================================================
